In [3]:
# Installing spotipy - You have to do it everytime if you are running the file in Colabs
! pip install spotipy 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import os
import sys
import pickle
import requests

In [0]:
def search_get_playlist_user(search_key,search_type = 'playlist'):

    ''' Gets User_id's from playlists extracted from a search key of choice

        Params : searck_key = Any key word you want to search with , E.g Blues,EDM,Romantic
                 search_type = The type of results we wantr to extract,E.g Playlists,Albums,Artists

        Returns : Created a file with all the unique user_id's extracted from the playlists which was extracted from the search    
    '''
    grant_type = 'client_credentials'
    body_params = {'grant_type' : grant_type}
    url='https://accounts.spotify.com/api/token'
    response = requests.post(url, data=body_params, auth = (Client_ID,Client_Secret)) 
    token_raw = json.loads(response.text)
    token = token_raw["access_token"]
    headers = {"Authorization": "Bearer {}".format(token),} 

    if os.path.exists('/content/drive/My Drive/spotify/users_id.txt'):
        with open("/content/drive/My Drive/spotify/users_id.txt", "rb") as fp: 
          old_data = pickle.load(fp)
          print('exists : ',len(old_data))

    else :
      old_data = ['spotify']

    lst = []
    

    url= "https://api.spotify.com/v1/search?q={}&type={}&limit=1&offset={}".format(search_key,search_type,0)
    r = requests.get(url, headers=headers)
    # print(json.dumps(r.json()['playlists']['items'][0]['owner']['id'],indent=4))
    total = int(r.json()['playlists']['total'])
    print(total)

    for i in range(0,total,50):
      url= "https://api.spotify.com/v1/search?q={}&type={}&limit=50&offset={}".format(search_key,search_type,i)
      r = requests.get(url, headers=headers)
      try:
        for item in r.json()['playlists']['items']:
          lst.append(item['owner']['id'])
      except Exception as e:
        pass

      pro_prec = float(i/total*100)
      sys.stdout.write("\r") # return to start of line
      sys.stdout.flush() 
      sys.stdout.write('Completed : {} %'.format(round(pro_prec,4)))


    print('\n',len(set(lst+list(old_data))))
    with open("/content/drive/My Drive/spotify/users_id.txt", "wb") as fp:
        pickle.dump(set(lst+list(old_data)),fp)


In [0]:
words_to_search = ["Jazz","Tamil","English","Hindi","party","Metal","Blue","Summer","Rain","Snow","EDM"]
for i in words_to_search[10]:
  #search_get_playlist_user(i,'playlist')
  import time
  #time.sleep(60)

In [0]:
def make_playlist_list(username,name):

    ''' Getting all Platlists for a give username '''

    Client_ID =  '9da29360dfe24b50bb9cf07341588d95'
    Client_Secret = '6869a24ff6c342ffb5f6409b758fe998'
    client_credentials_manager = SpotifyClientCredentials(Client_ID,Client_Secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    total = sp.user_playlists(username, limit=1, offset=0)['total']

    if os.path.exists("/content/drive/My Drive/spotify/playlists/"+name):
        with open("/content/drive/My Drive/spotify/playlists/"+name, "rb") as fp: 
            old_data = pickle.load(fp)
            #print('Exists :',len(old_data))
    else : 
        old_data = []

    lst = old_data

    for i in range(0, total, 50):
        a = sp.user_playlists(username, limit=50, offset=i)
        playlist_id = (a['items'])
        for j in range(0,len(playlist_id)):
            x = playlist_id[j]['id']
            ## print(x)
            lst.append(x)
    with open("/content/drive/My Drive/spotify/playlists/"+name, "wb") as fp:   #Pickling
        pickle.dump(lst, fp)
    
    #print('Number of Playlists id extracted :',len(lst))
    del lst


In [0]:
with open("/content/drive/My Drive/spotify/users_id.txt", "rb") as fp: 
  usernames = list(pickle.load(fp))
for i,username in enumerate(usernames):
  if i>= 28690 :
    prog_prec = i/len(usernames) * 100
    sys.stdout.flush() 
    sys.stdout.write("\r") # return to start of line
    sys.stdout.flush() 
    sys.stdout.write('Completed : {}% and number {} '.format(round(prog_prec,4),i))
    name = 'playlist_{}.txt'.format(i//1000)
    try:
      make_playlist_list(username,name)
    except KeyError:
      pass
    except Exception as e:
      if e.http_status == 429:
        print('API Rate Limit is excited ,Go sleep and hit the API after few hours')
        break;
      else:
        print('Error :',e.http_status ,' at ', i)
        pass

Completed : 34.0553% and number 33430 Error : 400  at  33430
Completed : 34.4332% and number 33801 

In [0]:
def track_ids(playlist_id,name):
  Client_ID =  '9da29360dfe24b50bb9cf07341588d95'
  Client_Secret = '6869a24ff6c342ffb5f6409b758fe998'
  grant_type = 'client_credentials'
  body_params = {'grant_type' : grant_type}
  url='https://accounts.spotify.com/api/token'
  response = requests.post(url, data=body_params, auth = (Client_ID,Client_Secret)) 
  token_raw = json.loads(response.text)
  token = token_raw["access_token"]
  headers = {"Authorization": "Bearer {}".format(token),} 

  if os.path.exists('/content/drive/My Drive/spotify/tracks/'+ name):
        with open("/content/drive/My Drive/spotify/tracks/" + name, "rb") as fp: 
          old_data = pickle.load(fp)
          #print('exists : ',len(old_data))
  else :
    old_data = ['spotify']
    
  lst = []
  url = 'https://api.spotify.com/v1/playlists/{}/tracks?fields=total&limit=1&offset={}'.format(playlist_id,0)
  r = requests.get(url, headers=headers)
  total = r.json()['total']

  for i in range(0,total,50):
      url= "https://api.spotify.com/v1/playlists/{}/tracks?fields=items&limit=50&offset={}".format(playlist_id,i)
      try:
        r = requests.get(url, headers=headers)
        items = r.json()['items']
        for item in items:
          lst.append(item['track']['id'])
      except Exception as e:
        print('Error :',e ,' at ', name)
        pass

  with open("/content/drive/My Drive/spotify/tracks/" + name, "wb") as fp:
    pickle.dump(set(lst+list(old_data)),fp)



In [63]:
for p_num in range(0,6): 
  with open("/content/drive/My Drive/spotify/playlist_{}.txt".format(p_num), "rb") as fp: 
    playlists = list(pickle.load(fp))
    for i,playlist in enumerate(playlists):
      if i>=0:
        prog_prec = i/len(usernames) * 100
        sys.stdout.flush() 
        sys.stdout.write("\r") # return to start of line
        sys.stdout.flush() 
        sys.stdout.write('Completed Playlist : {}% ,playList file : {} and number : {} '.format(round(prog_prec,4),p_num,i))
        name = 'track_id_{}'.format(i//10000)
        #track_ids(playlist,name)
  break;





In [14]:
def get_songs_details():

    with open('/content/drive/My Drive/spotify/track_id.txt',"rb") as fp:
        b = pickle.load(fp)
    print('Number of unique Songs id extracted : ',len(set(b)))

get_songs_details()

Number of unique Songs id extracted :  66556
